<a href="https://colab.research.google.com/github/nishithewel/ID5059GroupProject/blob/main/Feature_Engineering_Functions_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial setup to download dataset from KAggle.

In [ ]:
!pip install -q kaggle

Please follow the steps below to download and use kaggle data within Google Colab:

1. Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine

3. Upload the kaggle.json by running below code cell.

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anlihu","key":"69ad0d701447042b27eb0e12d8929cbe"}'}

In [ ]:
!rm -rf sample_data
!mkdir  -p /root/.kaggle/
!mv kaggle.json  /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!ls -lart /root/.kaggle/
!kaggle competitions download -c 'ieee-fraud-detection'
!mkdir data
!mv ieee-fraud-detection.zip data/
!unzip data/ieee-fraud-detection.zip -d data/

total 16
drwx------ 1 root root 4096 Apr  2 13:27 ..
-rw------- 1 root root   62 Apr  2 13:28 kaggle.json
drwxr-xr-x 2 root root 4096 Apr  2 13:28 .
 91% 108M/118M [00:00<00:00, 177MB/s] 
100% 118M/118M [00:00<00:00, 167MB/s]
mkdir: cannot create directory ‘data’: File exists
Archive:  data/ieee-fraud-detection.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_identity.csv  
  inflating: data/test_transaction.csv  
  inflating: data/train_identity.csv  
  inflating: data/train_transaction.csv  


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from scipy import stats
import gc

In [ ]:
train_identity, train_transaction = pd.read_csv('data/train_identity.csv'), pd.read_csv('data/train_transaction.csv')
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [ ]:
test_identity, test_transaction = pd.read_csv('data/test_identity.csv'),pd.read_csv('data/test_transaction.csv')
test_transaction.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train = pd.merge(train_transaction, train_identity, on = 'TransactionID', how = 'left')
test = pd.merge(test_transaction, test_identity, on = 'TransactionID', how = 'left')

In [ ]:
del train_identity, train_transaction, test_identity, test_transaction

In [ ]:
new_test_column = [col.replace('-','_') for col in test.columns.values]
test.columns = new_test_column

In [ ]:
all_features = pd.concat((train,test))

In [ ]:
# https://www.kaggle.com/code/yasagure/places-after-the-decimal-point-tell-us-a-lot/notebook
# multiply 1000 to see if there are three last digits
def getTransactionAmt_decimal(data):
  num = 3
  data = int(data * 1000)
  while(data % 10 == 0):
    num = num - 1
    data = data / 10
  if num < 0:
    num = 0
  return num


In [ ]:
# https://www.kaggle.com/code/davidcairuz/feature-engineering-lightgbm
def split_multivalues_columns(data):
  data[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = data['P_emaildomain'].str.split('.', expand=True)
  data[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = data['R_emaildomain'].str.split('.', expand=True)
    
  data['device_name'] = data['DeviceInfo'].str.split('/', expand=True)[0]
  data['device_version'] = data['DeviceInfo'].str.split('/', expand=True)[1]

  data['OS_id_30'] = data['id_30'].str.split(' ', expand=True)[0] 
  data['version_id_30'] = data['id_30'].str.split(' ', expand=True)[1]

  data['browser_id_31'] = data['id_31'].str.split(' ', expand=True)[0]
  data['version_id_31'] = data['id_31'].str.split(' ', expand=True)[1]

  data['screen_width'] = data['id_33'].str.split('x', expand=True)[0]
  data['screen_height'] = data['id_33'].str.split('x', expand=True)[1]

  # data['id_34'] = data['id_34'].str.split(':', expand=True)[1] these dont make sense,
  #  vals are array([nan, 'TRANSPARENT', 'ANONYMOUS', 'HIDDEN'], dtype=object)
  # data['id_23'] = data['id_23'].str.split(':', expand=True)[1]

  data.loc[data['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
  data.loc[data['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
  data.loc[data['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
  data.loc[data['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
  data.loc[data['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
  data.loc[data['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
  data.loc[data['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
  data.loc[data['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
  data.loc[data['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
  data.loc[data['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
  data.loc[data['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
  data.loc[data['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
  data.loc[data['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
  data.loc[data['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
  data.loc[data['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
  data.loc[data['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
  data.loc[data['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

  data.loc[data.device_name.isin(data.device_name.value_counts()[data.device_name.value_counts() < 200].index), 'device_name'] = "Others"
  gc.collect()
    
  return data  

In [ ]:
# https://www.kaggle.com/code/davidcairuz/feature-engineering-lightgbm
def aggregation(data):
  column_a = ['TransactionAmt', 'id_02', 'D15']
  column_b = ['card1', 'card4', 'addr1']

  for col_a in column_a:
    for col_b in column_b:
        data[f'{col_a}_to_mean_{col_b}'] = data[col_a] / data.groupby([col_b])[col_a].transform('mean')
        data[f'{col_a}_to_std_{col_b}'] = data[col_a] / data.groupby([col_b])[col_a].transform('std')

In [ ]:
def preprocess(all_features):
  # === multi values colums related ===
  all_features = split_multivalues_columns(all_features)

  # === TransactionAmt related ===
  all_features["TransactionAmt_decimal"] = all_features["TransactionAmt"].map(getTransactionAmt_decimal)
  all_features["TransactionAmt_cents"] = all_features["TransactionAmt"] - np.floor(all_features["TransactionAmt"]) 
  all_features["TransactionAmt_Log"] = np.log(all_features["TransactionAmt"])

  # === TransactionDT related ===
  all_features["Transaction_day_of_week"] = np.floor((all_features["TransactionDT"] / (3600 * 24) - 1) % 7)
  all_features["Transaction_hour"] = np.floor(all_features["TransactionDT"] / 3600) % 24

  # === aggregation related ===
  aggregation(all_features)

In [ ]:
# train_clean = all_features.iloc[:train.shape[0],:]
# test_clean = all_features.iloc[train.shape[0]:,:]